In [2]:
from datetime import datetime,date
from functools import wraps
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, ElementNotInteractableException
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, unquote

import csv
import hashlib
import logging
import os
import requests
import re
import time

from pathlib import Path
from typing import Tuple

In [3]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [4]:
comune = 'brindisi'
download_path = './ap'
url_comune = 'https://servizi.comune.brindisi.it/openweb/albo/albo_pretorio.php'

In [5]:
def retry(max_attempts=3, delay=10, exceptions=(Exception,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    attempts += 1
                    logging.info(f"Attempt {attempts} failed:", e)
                    time.sleep(delay)
            raise RuntimeError(f"Function {func.__name__} failed after {max_attempts} attempts")
        return wrapper
    return decorator

# Driver manager

In [6]:
class WebDriverManager:
    def __init__(self, DIR_DOWNLOAD):
        self.driver = None

    def start_driver(self, url):
        if not self.driver or (self.is_driver_open() == False):
            opts = Options()
            # start driver
            s = Service(r".\chromedriver.exe")
            prefs = {"download.default_directory": f'{download_path}',
                    "directory_upgrade": True,
                    "profile.default_content_settings.popups": 0}
            opts.add_experimental_option("prefs", prefs)
            self.driver = Chrome(service=s, options=opts)

            # opts = Options()
            # service = Service()

            # prefs = {"download.default_directory": f'{download_path}',
            #         "directory_upgrade": True,
            #         "profile.default_content_settings.popups": 0,
            #         "plugins.always_open_pdf_externally": True  # It will not show PDF directly in chrome
            #         }

            # opts.add_experimental_option('prefs', prefs)
            # opts.add_argument('--no-sandbox')
            # opts.add_argument('--headless')
            # opts.add_argument('--disable-gpu')
            # opts.add_argument('--disable-dev-shm-usage')

            # self.driver = Chrome(service=service,
            #                 options=opts)

            self.driver.get(url)
            # self.check_cookies()
            time.sleep(10) # Waiting for main page to load
        return self.driver  # Return the driver instance
        
    # No cookies button (?)
    # def check_cookies(self):
    #     try: 
    #         time.sleep(2)
    #         cookie_button = self.driver.find_element(By.CLASS_NAME, "btn.btn-primary.mr-2")

    #         cookie_button.click()
    #     except NoSuchElementException:
    #         logging.info(f'Cookies button was not found')
    #         pass
    #     except ElementNotInteractableException:
    #         pass

    def is_driver_open(self):
        if self.driver:
            try:
                # Access a property or method of the driver
                self.driver.current_url
                return True
            except NoSuchWindowException:
                return False
            except Exception as e:
                logging.info(f'Driver seems to be closed: {e}')
                return False
        return False
    
    def get_driver(self, link):
        if not self.is_driver_open():
            logging.warning(f'Driver is not responding, reopening')
            self.driver = self.start_driver(link)
        # else: 
        #     self.driver.get(link)
            # self.check_cookies()

        return self.driver

    def close_driver(self):
        if self.driver:
            self.driver.quit()
            self.driver = None

In [7]:
driver_manager = WebDriverManager(download_path)

In [8]:
driver = driver_manager.get_driver(url_comune)

# Loop through all pages of a section

In [20]:
body_e = driver.find_element(By.ID, 'tabella_albo')

rows_e = body_e.find_elements(By.CLASS_NAME, 'paginated_element')

In [24]:
for r_e in rows_e:
    # Find the div containing 'Data affissione:'
    data_affissione_div = r_e.find_element(By.XPATH, ".//div[label/text()='Data affissione:']")
    # data_affissione_text = data_affissione_div.text.split("Data affissione:")[1].strip()

    # Find the div containing 'Fine Pubblicazione:'
    fine_pubblicazione_div = r_e.find_element(By.XPATH, ".//div[label/text()='Fine Pubblicazione:']")
    # fine_pubblicazione_text = fine_pubblicazione_div.text.split("Fine Pubblicazione:")[1].strip()

    print(data_affissione_div.text, fine_pubblicazione_div.text)
    

05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 06/06/2024
05/06/2024 05/07/2024
05/06/2024 20/06/2024
05/06/2024 20/06/2024
05/06/2024 20/06/2024


In [31]:
while True:
    body_e = driver.find_element(By.ID, 'tabella_albo')

    rows_e = body_e.find_elements(By.CLASS_NAME, 'paginated_element')

    if rows_e:
        for r_e in rows_e:
            progressivo = r_e.find_element(By.TAG_NAME, 'div').text
            print(progressivo)
    else:
        break

    # Finding next page button:
    next_button = driver.find_element(By.XPATH, "//a[@class='button' and @title='Successiva']")
    # Clicking on it using js
    # driver.execute_script("arguments[0].click();", next_button)
    next_button.click()

2024/0002715
2024/0002714
2024/0002713
2024/0002712
2024/0002711
2024/0002710
2024/0002709
2024/0002708
2024/0002707
2024/0002706
2024/0002705
2024/0002704
2024/0002703
2024/0002702
2024/0002701


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (603, 5574)
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF6D9451F22+60322]
	(No symbol) [0x00007FF6D93CCE99]
	(No symbol) [0x00007FF6D9287EBA]
	(No symbol) [0x00007FF6D92DF32E]
	(No symbol) [0x00007FF6D92DCCF2]
	(No symbol) [0x00007FF6D92DA18B]
	(No symbol) [0x00007FF6D92D9356]
	(No symbol) [0x00007FF6D92CB491]
	(No symbol) [0x00007FF6D92FC21A]
	(No symbol) [0x00007FF6D92CADB6]
	(No symbol) [0x00007FF6D92FC430]
	(No symbol) [0x00007FF6D931BC80]
	(No symbol) [0x00007FF6D92FBFC3]
	(No symbol) [0x00007FF6D92C9617]
	(No symbol) [0x00007FF6D92CA211]
	GetHandleVerifier [0x00007FF6D976946D+3301613]
	GetHandleVerifier [0x00007FF6D97B3693+3605267]
	GetHandleVerifier [0x00007FF6D97A9410+3563664]
	GetHandleVerifier [0x00007FF6D95042F6+790390]
	(No symbol) [0x00007FF6D93D74DF]
	(No symbol) [0x00007FF6D93D33D4]
	(No symbol) [0x00007FF6D93D3562]
	(No symbol) [0x0

In [12]:
# selector for page number
page_s_e = driver.find_element(By.ID, 'range_pagina')

pages_e = page_s_e.find_elements(By.TAG_NAME, 'option')

pages = [p.get_attribute('value') for p in pages_e if p.get_attribute('value') != '']

for p in pages:
    print(f'Getting elements: {p}')

    page_s_e = driver.find_element(By.ID, 'range_pagina')

    page_select = Select(page_s_e)
    
    page_select.select_by_value(p)

    time.sleep(2)


Getting elements: 1-10
Getting elements: 11-12


# Getting oggetti and info

In [29]:
body_e = driver.find_element(By.CLASS_NAME, 'it-list-wrapper')

oggetti_e = body_e.find_elements(By.XPATH, './ul/li')

In [30]:
len(oggetti_e)

2

In [32]:
for o_e in oggetti_e:
    print(F'NEW_OGGETTO')
    ul_e = o_e.find_element(By.CLASS_NAME, 'link-sublist')
    print(ul_e.text)

NEW_OGGETTO
Oggetto Pubblicazione di Matrimonio di SCARAMUCCI Daniele e VANNINI Ilenia
In pubblicazione dal 28/05/2024 al 05/06/2024
Registro: 2024/3237
Ente proponente: COMUNE DI AREZZO UFFICIO STATO CIVILE
NEW_OGGETTO
Oggetto Pubblicazione di Matrimonio di BRACALENTI Massimo e SBRAGI Michela
In pubblicazione dal 29/05/2024 al 06/06/2024
Registro: 2024/3247
Ente proponente: COMUNE DI AREZZO UFFICIO STATO CIVILE


In [39]:
info_e = driver.find_element(By.CLASS_NAME, 'link-sublist')
print(info_e.text)

# Regular expression pattern to match the date pattern
date_pattern = r"In pubblicazione dal (\d{2}/\d{2}/\d{4}) al (\d{2}/\d{2}/\d{4})"

# Find all date patterns in the info
date_matches = re.search(date_pattern, info_e.text)

if date_matches:
    start_date = datetime.strptime(date_matches.group(1), '%d/%m/%Y')
    end_date = datetime.strptime(date_matches.group(2), '%d/%m/%Y')
else:
    print('No dates were found')

Oggetto Pubblicazione di Matrimonio di SCARAMUCCI Daniele e VANNINI Ilenia
In pubblicazione dal 28/05/2024 al 05/06/2024
Registro: 2024/3237
Ente proponente: COMUNE DI AREZZO UFFICIO STATO CIVILE


In [41]:
start_date, end_date

(datetime.datetime(2024, 5, 28, 0, 0), datetime.datetime(2024, 6, 5, 0, 0))

In [42]:
atto_pattern = r"Registro: 2024/(\d+)"

# Find matches in the string
match = re.search(atto_pattern, info_e.text)
print(match.group(1))

3237


# Downloading files

In [100]:
body_e = driver.find_element(By.CLASS_NAME, 'documento')
print(body_e.text)

Pubblicazione di Matrimonio di BRACALENTI Massimo e SBRAGI Michela
Registro: 2024/3247
Tipologia: Pubbl.Matrimonio ordinaria
In pubblicazione dal 29/05/2024 al 06/06/2024
Ente proponente: COMUNE DI AREZZO UFFICIO STATO CIVILE - VerbalePubblicazioneMatrimonioNumero:2024/149
Allegati:
Pubblicazioni di matrimonio BRACALENTI Massimo e SBRAGI Michela - Pubblicazioni di matrimonio BRACALENTI Massimo e SBRAGI Michela.pdf.p7m - 385,70 Kb


In [101]:
a_elements = body_e.find_elements(By.TAG_NAME, 'a')

In [102]:
for a_e in a_elements:
    sibling_text = driver.execute_script("""
    var a = arguments[0];
    var text = a.nextSibling.nodeValue;
    return text;
    """, a_e).strip()

    print(sibling_text)

- Pubblicazioni di matrimonio BRACALENTI Massimo e SBRAGI Michela.pdf.p7m - 385,70 Kb


# Class oggetti Ricerca

In [122]:
class SectionsAlbo:
    def __init__(self, driver_manager: WebDriverManager, section_otion: str, dir_download: str, last_date_update: datetime):
        self.driver_manager = driver_manager
        self.driver = self.get_section(section_option=section_otion)
        self.dir_download = self.check_dir_download(dir_download)
        self.last_date_update = last_date_update
        self.dict_links_oggetti = self.loop_all_pages()

    def get_section(self, section_option: str):
        driver = self.driver_manager.get_driver(url_comune)

        menu_e = driver.find_element(By.ID, 'tendinaTipoPratiche')
        select_menu = Select(menu_e)

        select_menu.select_by_value(section_option)

        button_cerca = driver.find_element(By.ID, 'inforBottoneConfermaAlbo')
        button_cerca.click()
        return driver
        
    def check_dir_download(self, dir_download):
        # Creating directory if does not exist:
        if not os.path.exists(dir_download):
            os.makedirs(dir_download)
        return dir_download

    def get_n_atto(self, info_atto: str)->str:
        try:
            atto_pattern = r"Registro: 2024/(\d+)"

            # Find matches in the string
            match = re.search(atto_pattern, info_atto)
            return match.group(1)
        except Exception as e:
            logging.warning(f'SectionsAlbo: Could not get n_atto: {e}')
    
    def get_publish_dates(self, info_atto: str)->Tuple[datetime, datetime]:
        try:
            # Regular expression pattern to match the date pattern
            date_pattern = r"In pubblicazione dal (\d{2}/\d{2}/\d{4}) al (\d{2}/\d{2}/\d{4})"

            # Find all date patterns in the info
            date_matches = re.search(date_pattern, info_atto)

            start_date = datetime.strptime(date_matches.group(1), '%d/%m/%Y')
            end_date = datetime.strptime(date_matches.group(2), '%d/%m/%Y')

            return start_date, end_date
        except Exception as e:
            logging.warning(f'SectionsAlbo: It was not possible to get publish dates: {e}')
            return None, None
    
    def get_oggetto_link(self, ogg_e: WebElement)->str:
        try:
            a_element = ogg_e.find_element(By.TAG_NAME, 'a')
            return a_element.get_attribute('href')

        except Exception as e:
            logging.warning(f'SectionsAlbo: Could not get link from oggetto {e}')
        pass

    @staticmethod
    def rename_file(file_name:str, link_ogg: str)->str:
        # Generate a unique identifier for the file
        id = hashlib.sha256(link_ogg.encode()).hexdigest()[:8]

        return f'{id}_{file_name}'

    def download_file(self, file_name:str, link_ogg:str)->Path:
        try: 
            # Download the file
            response = requests.get(link_ogg)

            # Construct the full path of the file
            file_path = Path(self.dir_download, file_name)

            # Save the file
            with open(file_path, 'wb') as f:
                f.write(response.content)
            
            return file_path
        except Exception as e:
            logging.warning(f'SectionsAlbo: It was not possible to download file: {e}')  
        pass

    def get_atti_from_page(self) -> dict:
        try:
            all_oggetti_section = {}
            body_e = self.driver.find_element(By.CLASS_NAME, 'it-list-wrapper')

            oggetti_e  = body_e.find_elements(By.XPATH, './ul/li')

            for ogg in oggetti_e:
                info_atto = ogg.find_element(By.CLASS_NAME, 'link-sublist').text
                start_date, _ = self.get_publish_dates(info_atto=info_atto)
                if (self.last_date_update == None) or (self.last_date_update < start_date):
                    dict_ogg = self.get_main_info_oggetto(oggetto=ogg, 
                                                          info_atto=info_atto)

                    all_oggetti_section.update(dict_ogg)
            
            return all_oggetti_section
        except Exception as e:
            logging.warning(f'SectionsAlbo: Could not get the atti from this page: {e}')

    def loop_all_pages(self) -> dict:
        try: 
            all_oggetti_dict = {}
            # selector for page number to get all options
            page_s_e = self.driver.find_element(By.ID, 'range_pagina')

            pages_e = page_s_e.find_elements(By.TAG_NAME, 'option')

            pages = [p.get_attribute('value') for p in pages_e if p.get_attribute('value') != '']

            for p in pages:
                # Finding page selector in the given page
                page_s_e = driver.find_element(By.ID, 'range_pagina')

                # Using selenium create a select menu with each element
                page_select = Select(page_s_e)
                
                # Selecting each page
                page_select.select_by_value(p)

                time.sleep(2)

                dict_section = self.get_atti_from_page()
                all_oggetti_dict.update(dict_section)

            return all_oggetti_dict

        except NoSuchElementException:
            pass
            return {}
        except Exception as e:
            logging.warning(f'OggetiRicercaAlbo: Could not get all page numbers: {e}')
            return {}

    def get_main_info_oggetto(self, oggetto: WebElement, info_atto: str) -> dict:
        n_atto = self.get_n_atto(info_atto=info_atto)
        start_date, end_date = self.get_publish_dates(info_atto=info_atto)
        link = self.get_oggetto_link(ogg_e=oggetto)
        
        return {link: dict(n_atto=n_atto,
                           start_date=start_date,
                           end_date=end_date)}

    def get_dict_objects(self):
        dict_ogg = {}
        logging.info(f'Found {len(self.dict_links_oggetti)} oggetti')
        for link, dict_info in self.dict_links_oggetti.items():
            self.driver.get(link)

            body_e = self.driver.find_element(By.CLASS_NAME, 'documento')
            files_e = body_e.find_elements(By.TAG_NAME, 'a')

            for f_e in files_e:
                # Getting the sibling text where we can find the file name
                sibling_text = driver.execute_script("""
                                                     var a = arguments[0];
                                                     var text = a.nextSibling.nodeValue;
                                                     return text;
                                                     """, f_e).strip()

                pattern_f_name = r'- (.*?) -'

                # Find the file name
                match = re.search(pattern_f_name, sibling_text)
                
                f_name = match.group(1)
                f_link = f_e.get_attribute('href')

                file_name = self.rename_file(f_name, f_link)

                # Update dict_info with new keys and values
                updated_dict_info = dict_info.copy()
                updated_dict_info.update(dict(file_name=file_name,
                                              link=f_link,
                                              internal_path=self.download_file(file_name=file_name, link_ogg=f_link)))

                dict_ogg[f_name] = updated_dict_info

        return dict_ogg


In [119]:
sections_manager = SectionsAlbo(driver_manager=driver_manager,
                                section_otion='MAT01',
                                dir_download=download_path,
                                last_date_update=None)

In [120]:
sections_manager.dict_links_oggetti

{'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=dettaglio&idPratica=114186&': {'n_atto': '3060',
  'start_date': datetime.datetime(2024, 5, 23, 0, 0),
  'end_date': datetime.datetime(2024, 5, 31, 0, 0)},
 'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=dettaglio&idPratica=114267&': {'n_atto': '3139',
  'start_date': datetime.datetime(2024, 5, 23, 0, 0),
  'end_date': datetime.datetime(2024, 5, 31, 0, 0)},
 'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=dettaglio&idPratica=114268&': {'n_atto': '3140',
  'start_date': datetime.datetime(2024, 5, 23, 0, 0),
  'end_date': datetime.datetime(2024, 5, 31, 0, 0)},
 'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=dettaglio&idPratica=114275&': {'n_atto': '3147',
  'start_date': datetime.datetime(2024, 5, 24, 0, 0),
  'end_date': datetime.datetime(2024, 6, 1, 0, 0)},
 'https://servizionline.comune.arezzo.it/jalbopretorionew

In [121]:
sections_manager.get_dict_objects()

INFO:root:Found 12 oggetti


- Matrimonio SCARANO Roberto e ACCIAI Giulia.pdf.p7m - 363,08 Kb
- Pubblicazioni matrimonio DEL PACE Andrea e POMARANZI Nicla.pdf.p7m - 386,11 Kb
- noferi.nardone.pdf.p7m - 48,18 Kb
- Pubblicazioni di Matrimonio PROFICE Francesco e PICCINELLI Anna Emma.pdf.p7m - 374,36 Kb
- pubbl.di matrimoni odi PERRONE e IANIRI.pdf.p7m - 462,48 Kb
- Pubblicazioni matrimonio MAZZOLI David e MEARINI Patricia.pdf.p7m - 379,64 Kb
- Pubblicazioni di Matrimonio GALLI Amerigo e PAGGINI Marzia.pdf.p7m - 376,46 Kb
- Pubblicazioni di matrimonio SANTINELLI Mauro e CHECCARINI Paola.pdf.p7m - 441,34 Kb
- Pubblicazioni di Matrimonio BEGLIOMINI Alberto e COMANDUCCI Ilaria.pdf.p7m - 379,98 Kb
- Pubblicazioni di Matrimonio BALDI Daniele e PELUCCHINI Sara.pdf.p7m - 370,45 Kb
- PUBBLICAZ. DI MATRIMONI ODI SCARAMUCCI E VANNACCI.pdf.p7m - 399,01 Kb
- Pubblicazioni di matrimonio BRACALENTI Massimo e SBRAGI Michela.pdf.p7m - 385,70 Kb


{'Matrimonio SCARANO Roberto e ACCIAI Giulia.pdf.p7m': {'n_atto': '3060',
  'start_date': datetime.datetime(2024, 5, 23, 0, 0),
  'end_date': datetime.datetime(2024, 5, 31, 0, 0),
  'file_name': '322c443a_Matrimonio SCARANO Roberto e ACCIAI Giulia.pdf.p7m',
  'link': 'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=allegato&idDocumentale=316319',
  'internal_path': WindowsPath('ap/322c443a_Matrimonio SCARANO Roberto e ACCIAI Giulia.pdf.p7m')},
 'Pubblicazioni matrimonio DEL PACE Andrea e POMARANZI Nicla.pdf.p7m': {'n_atto': '3139',
  'start_date': datetime.datetime(2024, 5, 23, 0, 0),
  'end_date': datetime.datetime(2024, 5, 31, 0, 0),
  'file_name': 'e0a24858_Pubblicazioni matrimonio DEL PACE Andrea e POMARANZI Nicla.pdf.p7m',
  'link': 'https://servizionline.comune.arezzo.it/jalbopretorionew/AlboPretorio?servizio=allegato&idDocumentale=316585',
  'internal_path': WindowsPath('ap/e0a24858_Pubblicazioni matrimonio DEL PACE Andrea e POMARANZI Nicla.pdf.p7m'

In [90]:
def get_all_sections(driver_manager: WebDriverManager,
                     url_comune: str) -> dict:
    
    driver = driver_manager.get_driver(url_comune)

    # find select menu
    select_menu_e = driver.find_element(By.ID, 'tendinaTipoPratiche')

    options_e = select_menu_e.find_elements(By.TAG_NAME, 'option')

    options = [o.get_attribute('value') for o in options_e if o.get_attribute('value') != '']
    sections_names = [o.text.strip() for o in options_e if o.text.strip() != 'Tutti']

    return dict(map(lambda i,j : (i,j) , sections_names,options))
    

In [91]:
sections = get_all_sections(driver_manager=driver_manager,
                            url_comune=url_comune)

In [123]:
sections

{'Atti del Consiglio Comunale': 'CC',
 'Atti della Giunta Municipale': 'GC',
 'Autorizzazione': 'X02',
 'Avviso': 'X03',
 'Concessioni e Autorizzazioni': 'PM',
 'Decreti del Sindaco': 'SD',
 'Deliberazione di altro Ente': 'X09',
 'Determinazioni Dirigenziali': 'D',
 'Elenco': 'X11',
 'Notifica': 'X13',
 'Ordinanze del Dirigente': 'DO',
 'Ordinanze del Sindaco': 'SO',
 'Pubbl.Matrimonio ordinaria': 'MAT01'}

In [124]:
for s, opt in sections.items():
    print(s)
    dir_download_section = f'{download_path}/{s}'
    SectionsAlbo(driver_manager=driver_manager,
                 section_otion=opt,
                 dir_download=dir_download_section,
                 last_date_update=None).get_dict_objects()

Atti del Consiglio Comunale


INFO:root:Found 6 oggetti


Atti della Giunta Municipale


INFO:root:Found 48 oggetti


Autorizzazione


...

INFO:root:Found 72 oggetti


Ordinanze del Sindaco


INFO:root:Found 2 oggetti


Pubbl.Matrimonio ordinaria


INFO:root:Found 12 oggetti
